In [15]:
sample_or_prod = True #True = sample, False = production

batch_size = 64 #ajust to suit memory capacity of hardware.

In [16]:
HOMEPATH = "/home/ubuntu/fastai/"

DATA_PATH = HOMEPATH+"data/Kaggle_dogs-vs-cats-redux-kernels-edition/"

MODEL_PATH = DATA_PATH+"models/"
RESULTS_PATH = DATA_PATH+"results/"


SAMPLE_DATA_PATH = DATA_PATH + "sample/"#choose this for testing or above for production
SAMPLE_TEST_PATH = SAMPLE_DATA_PATH+"test/"
SAMPLE_TRAIN_PATH = SAMPLE_DATA_PATH + "train/"
SAMPLE_VALID_PATH = SAMPLE_DATA_PATH + "valid/"


TEST_PATH = DATA_PATH+"test/"
TRAIN_PATH = DATA_PATH + "train/"
VALID_PATH = DATA_PATH + "valid/"


if sample_or_prod:
    WORKING_DATA  = SAMPLE_DATA_PATH
    WORKING_TEST  = SAMPLE_TEST_PATH
    WORKING_TRAIN = SAMPLE_TRAIN_PATH
    WORKING_VALID = SAMPLE_VALID_PATH
    s_or_p = "_sample_"
else:
    WORKING_DATA  = DATA_PATH
    WORKING_TEST  = TEST_PATH
    WORKING_TRAIN = TRAIN_PATH
    WORKING_VALID = VALID_PATH
    s_or_p = "_prod_"

    
print "WORKING_DATA:", WORKING_DATA
print "WORKING_TEST:", WORKING_TEST
print "WORKING_TRAIN:", WORKING_TRAIN
print "WORKING_VALID:", WORKING_VALID
print "s_or_p:", s_or_p


WORKING_DATA: /home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/
WORKING_TEST: /home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/test/
WORKING_TRAIN: /home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/train/
WORKING_VALID: /home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/valid/
s_or_p: _sample_


In [17]:
import os
from datetime import datetime
from glob import glob
from random import shuffle
from shutil import copyfile


In [18]:
os.chdir(HOMEPATH)
print ("os.getcwd:", os.getcwd())
# Rather than importing everything manually, we'll make things easy
#   and load them all in utils.py, and just import them from there.
%matplotlib inline
import utils; reload(utils)
from utils import *


('os.getcwd:', '/home/ubuntu/fastai')


In [19]:
os.chdir(DATA_PATH)
print (os.getcwd())
!tree -d

/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition
.
├── models
│   ├── train_convlayer_features_prod_.bc
│   │   ├── data
│   │   └── meta
│   ├── train_convlayer_features_sample_.bc
│   │   ├── data
│   │   └── meta
│   ├── train_data_prod_.bc
│   │   ├── data
│   │   └── meta
│   ├── train_data_sample_.bc
│   │   ├── data
│   │   └── meta
│   ├── valid_convlayer_features_prod_.bc
│   │   ├── data
│   │   └── meta
│   ├── valid_convlayer_features_sample_.bc
│   │   ├── data
│   │   └── meta
│   ├── valid_data_prod_.bc
│   │   ├── data
│   │   └── meta
│   └── valid_data_sample_.bc
│       ├── data
│       └── meta
├── results
├── sample
│   ├── test
│   │   └── unknown
│   ├── train
│   │   ├── cat
│   │   └── dog
│   └── valid
│       ├── cat
│       └── dog
├── test
│   └── unknown
├── train
│   ├── cat
│   └── dog
└── valid
    ├── cat
    └── dog

43 directories


In [20]:
CATEGORIES = ['cat/', 'dog/']#nb: do not change order or trailing slash
UNKNOWN = 'unknown/'


In [21]:
DATA_DIR_LIST = [DATA_PATH, SAMPLE_DATA_PATH, SAMPLE_TEST_PATH, SAMPLE_TEST_PATH+UNKNOWN, SAMPLE_TRAIN_PATH, 
                 SAMPLE_VALID_PATH, TEST_PATH, TEST_PATH+UNKNOWN, TRAIN_PATH, VALID_PATH, MODEL_PATH, RESULTS_PATH]



for category in CATEGORIES:
    DATA_DIR_LIST.append(TRAIN_PATH+category)
    DATA_DIR_LIST.append(VALID_PATH+category)
    DATA_DIR_LIST.append(SAMPLE_TRAIN_PATH+category)
    DATA_DIR_LIST.append(SAMPLE_VALID_PATH+category)


#delete the variables we don't need to prevent accidental use.
del SAMPLE_DATA_PATH
del SAMPLE_TEST_PATH
del SAMPLE_TRAIN_PATH
del SAMPLE_VALID_PATH
del TEST_PATH
del TRAIN_PATH
del VALID_PATH


In [22]:
def dirFileList(dir_path):
    return [name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))]

In [23]:
def listDirsFileCount(DATA_DIR_LIST):
    DATA_DIR_LIST = sorted(DATA_DIR_LIST)
    for dir_ in DATA_DIR_LIST:
        print dir_, len(dirFileList(dir_))


In [24]:
def showLayersInfo(model):
    print ("Number of layers : ", len(model.layers))
    count = 0
    for layer in model.layers:
        print count, type(layer), ", trainable:", layer.trainable
        print "input:", layer.input_shape, ", output:",layer.output_shape, "\n"
        count +=1

In [25]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

In [26]:
def plot_history(history):
    # list all data in history
    print(history.history.keys())
    # summarize history for accuracy
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validate'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validate'], loc='upper left')
    plt.show()

In [27]:
listDirsFileCount(DATA_DIR_LIST)

/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/ 3
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/models/ 13
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/results/ 1
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/ 0
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/test/ 0
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/test/unknown/ 2500
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/train/ 0
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/train/cat/ 1750
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/train/dog/ 1750
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/valid/ 0
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/valid/cat/ 750
/home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/valid/dog/ 75

In [29]:
trn_batches = get_batches(WORKING_TRAIN, shuffle=False, batch_size=batch_size)
val_batches = get_batches(WORKING_VALID, shuffle=False, batch_size=batch_size)

Found 3500 images belonging to 2 classes.
Found 1500 images belonging to 2 classes.


In [35]:
print "WORKING_DATA:", WORKING_DATA
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(WORKING_DATA)
#utils.get_classes
#return (val_batches.classes, batches.classes, onehot(val_batches.classes), onehot(batches.classes),
#        val_batches.filenames, batches.filenames, test_batches.filenames)
#NB: get_classes expects data in subdirectories 'train', 'valid', 'test'
print "val_classes:", type(val_classes), val_classes.shape, val_classes[0:5]
print "trn_classes:", type(trn_classes), trn_classes.shape, trn_classes[0:5]
print "val_labels:", type(val_labels), val_labels.shape, "\n", val_labels[0:5]
print "trn_labels:", type(trn_labels), trn_labels.shape, "\n",trn_labels[0:5]
print "val_filenames:", type(val_filenames), len(val_filenames)
print "filenames:", type(filenames), len(filenames)
print "test_filenames:", type(test_filenames), len(test_filenames)

#NBB: val_labels = onehot(val_classes) & trn_labels = onehot(trn_classes). onehot 'converts' from 1 to [0,1]

WORKING_DATA: /home/ubuntu/fastai/data/Kaggle_dogs-vs-cats-redux-kernels-edition/sample/
Found 3500 images belonging to 2 classes.
Found 1500 images belonging to 2 classes.
Found 2500 images belonging to 1 classes.
val_classes: <type 'numpy.ndarray'> (1500,) [0 0 0 0 0]
trn_classes: <type 'numpy.ndarray'> (3500,) [0 0 0 0 0]
val_labels: <type 'numpy.ndarray'> (1500, 2) 
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
trn_labels: <type 'numpy.ndarray'> (3500, 2) 
[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
val_filenames: <type 'list'> 1500
filenames: <type 'list'> 3500
test_filenames: <type 'list'> 2500


In [36]:
model = Vgg16().model
#model is stock vgg16 model, all layers trainable, input [3,224,224], output 1,000 classes. needs to be modified.
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [37]:
showLayersInfo(model)

('Number of layers : ', 38)
0 <class 'keras.layers.core.Lambda'> , trainable: True
input: (None, 3, 224, 224) , output: (None, 3, 224, 224) 

1 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 3, 224, 224) , output: (None, 3, 226, 226) 

2 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 3, 226, 226) , output: (None, 64, 224, 224) 

3 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 64, 224, 224) , output: (None, 64, 226, 226) 

4 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 64, 226, 226) , output: (None, 64, 224, 224) 

5 <class 'keras.layers.pooling.MaxPooling2D'> , trainable: True
input: (None, 64, 224, 224) , output: (None, 64, 112, 112) 

6 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 64, 112, 112) , output: (None, 64, 114, 114) 

7 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input

In [38]:
conv_layers,fc_layers = split_at(model, Convolution2D)
#utils.split_at(model, layer_type) 
#splits model at last occurrance of layer_type. (in this case Convolution2D)


In [41]:
conv_model = Sequential(conv_layers)
#31 layers, all trainable, last layer = Convolution2D
showLayersInfo(conv_model)

('Number of layers : ', 31)
0 <class 'keras.layers.core.Lambda'> , trainable: True
input: (None, 3, 224, 224) , output: (None, 3, 224, 224) 

1 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 3, 224, 224) , output: (None, 3, 226, 226) 

2 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 3, 226, 226) , output: (None, 64, 224, 224) 

3 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 64, 224, 224) , output: (None, 64, 226, 226) 

4 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input: (None, 64, 226, 226) , output: (None, 64, 224, 224) 

5 <class 'keras.layers.pooling.MaxPooling2D'> , trainable: True
input: (None, 64, 224, 224) , output: (None, 64, 112, 112) 

6 <class 'keras.layers.convolutional.ZeroPadding2D'> , trainable: True
input: (None, 64, 112, 112) , output: (None, 64, 114, 114) 

7 <class 'keras.layers.convolutional.Convolution2D'> , trainable: True
input

In [40]:
showLayersInfo(Sequential(fc_layers))
#7 layers, all trainable, last layer = Dence, 1000 classes output.

('Number of layers : ', 7)
0 <class 'keras.layers.pooling.MaxPooling2D'> , trainable: True
input: (None, 512, 14, 14) , output: (None, 512, 7, 7) 

1 <class 'keras.layers.core.Flatten'> , trainable: True
input: (None, 512, 7, 7) , output: (None, 25088) 

2 <class 'keras.layers.core.Dense'> , trainable: True
input: (None, 25088) , output: (None, 4096) 

3 <class 'keras.layers.core.Dropout'> , trainable: True
input: (None, 4096) , output: (None, 4096) 

4 <class 'keras.layers.core.Dense'> , trainable: True
input: (None, 4096) , output: (None, 4096) 

5 <class 'keras.layers.core.Dropout'> , trainable: True
input: (None, 4096) , output: (None, 4096) 

6 <class 'keras.layers.core.Dense'> , trainable: True
input: (None, 4096) , output: (None, 1000) 



In [43]:
startTime= datetime.now()
print "startTime:", startTime

val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
print "val_features:", val_features.shape
trn_features = conv_model.predict_generator(trn_batches, trn_batches.nb_sample)
print "trn_features:", trn_features.shape

timeElapsed=datetime.now()-startTime
print 'Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed)

#https://keras.io/models/sequential/
#predict_generator(self, generator, steps=None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
#Returns: A Numpy array of predictions.

('startTime:', datetime.datetime(2017, 12, 20, 2, 1, 49, 914114))
val_features: (1500, 512, 14, 14)
trn_features: (3500, 512, 14, 14)
Time elapsed (hh:mm:ss.ms) 0:02:05.674847


In [ ]:
#save_array(MODEL_PATH + 'train_convlayer_features.bc', trn_features)
#save_array(MODEL_PATH + 'valid_convlayer_features.bc', val_features)

In [44]:
#trn_features = load_array(MODEL_PATH+'train_convlayer_features.bc')
#val_features = load_array(MODEL_PATH+'valid_convlayer_features.bc')

In [ ]:
startTime= datetime.now()
print "startTime:", startTime

trn = get_data(WORKING_TRAIN)
print("trn:", trn.shape)
val = get_data(WORKING_VALID)
print("val:", val.shape)

timeElapsed=datetime.now()-startTime
print 'Time elapsed (hh:mm:ss.ms) {}'.format(timeElapsed)
